In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd
import numpy as np
import xarray as xr
import plotly.graph_objects as go
from tqdm import tqdm
from os.path import join as pjoin
from scipy.ndimage import gaussian_filter
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence

sys.path.append('/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis')
import circletrack_neural as ctn
import circletrack_behavior as ctb
import place_cells as pc
import plotting_functions as pf

In [ ]:
project_dir = 'MultiCon_Imaging'
experiment_dir = 'MultiCon_Imaging5'
mouse_list = ['mc44', 'mc46', 'mc48']
dpath = f'../../{project_dir}/{experiment_dir}/output/aligned_minian/'
spath = f'../../{project_dir}/{experiment_dir}/output/aligned_place_cells/'
fig_path = f'../../../Manuscripts/MultiCon/intermediate_plots/si_across_trials'
if not os.path.exists(fig_path):
    os.makedirs(fig_path)
data_type = 'S'
only_running = True
correct_dir = True
velocity_thresh = 10
bin_size = 0.16 ## 2cm wide bins in radians
min_event_amount = 0.2
rs = RandomState(MT19937(SeedSequence(24601)))

### Single mouse - get spatial information of each cell across trials.

In [ ]:
mtest = 'mc52'
si_dict = {'mouse': [], 'sex': [], 'group': [], 'session': [], 'trial': [], 'unit_id': [], 'si': []}
for mouse in [mtest]:
    mpath = pjoin(dpath, f'{mouse}/{data_type}')
    # for session in os.listdir(mpath):
    for session in [f'{mtest}_S_5.nc']:
        sdata = xr.open_dataset(pjoin(mpath, session))[data_type]
        num_neurons = sdata.shape[0]
        minimum_act_bool = pc.minimum_activity_level(sdata, minimum_event_amount=min_event_amount, bin_size_seconds=60, fps=30, func=np.sum, binarized=0)
        sdata = sdata[:, sdata['behav_t'] >= 5] ## remove the first five seconds
        forward, reverse = ctb.get_forward_reverse_trials(sdata, percent_correct=63)
        for ftrial in forward:
            sub_data = sdata[:, sdata['trials'] == ftrial]
            pos_data = gaussian_filter(sub_data['lin_position'].values, 2) ## filter width of two
            population_activity, occupancy, _ = pc.spatial_activity(sub_data.values, pos_data, bin_size=bin_size, binarized=False)
            tuning_curves = pc.get_tuning_curves(population_activity, occupancy)
            bits_per_event = pc.skaggs_information_content(tuning_curves, occupancy)
            for idx in np.arange(0, sub_data.shape[0]):
                si_dict['mouse'].append(mouse)
                si_dict['sex'].append(sdata.attrs['sex'])
                si_dict['group'].append(sdata.attrs['group'])
                si_dict['session'].append(sdata.attrs['session_two'])
                si_dict['trial'].append(ftrial)
                si_dict['unit_id'].append(sdata['unit_id'].values[idx])
                si_dict['si'].append(bits_per_event[idx])
si_df = pd.DataFrame(si_dict)

In [ ]:
uids = np.sort(rs.randint(low=0, high=sdata.shape[0], size=10))
avg_si = si_df.groupby(['mouse', 'session', 'trial'], as_index=False).agg({'si': ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='Trial', y_title='Spatial Information (bits/event)')
for uid in uids:
    fig.add_trace(go.Scattergl(x=si_df['trial'].unique(), y=si_df['si'][si_df['unit_id'] == uid], name=f'{uid}'))
fig.add_trace(go.Scattergl(x=avg_si['trial'], y=avg_si['si']['mean'], line_color='black', name='Average', marker_line_width=12))
fig.show()
fig.write_image(pjoin(fig_path, f'{mouse}_avg_si_across_trials_with_random_cells.png'))